## Profile upserts to grapher

Notebook with line_profiler for optimizing upserting to grapher.

In [1]:
from owid.catalog import Dataset, Table
from etl import paths

%load_ext autoreload
%autoreload 2
%load_ext line_profiler

In [2]:
step = 'iea/2024-07-04/critical_minerals_demand_by_technology'
table_name = 'demand_by_technology'

ds = Dataset(paths.DATA_DIR / "grapher" / step)
table = ds[table_name]
table.head()

demand_graphite__battery_grade__refinery_base_case_announced_pledges  \
country           year                                                                         
Electric vehicles 2023                                             685564                      
                  2030                                            3212350                      
                  2035                                            4865630                      
                  2040                                            5157100                      
                  2045                                            3990620                      

                        demand_graphite__battery_grade__refinery_base_case_net_zero_by_2050  \
country           year                                                                        
Electric vehicles 2023                                             685564                     
                  2030                                            4537460                     
                  2035                                            6083980                     
                  2040                                            5777930                     
                  2045                                            4660410                     

                        demand_graphite__battery_grade__refinery_base_case_stated_policies  \
country           year                                                                       
Electric vehicles 2023                                             685564                    
                  2030                                            2791720                    
                  2035                                            3825600                    
                  2040                                            3674590                    
                  2045                                            2768200                    

                        demand_cobalt_refinery_base_case_announced_pledges  \
country           year                                                       
Electric vehicles 2023                                       61840.902344    
                  2030                                           171264.0    
                  2035                                           202802.0    
                  2040                                      259971.015625    
                  2045                                           286273.0    

                        demand_cobalt_refinery_base_case_net_zero_by_2050  \
country           year                                                      
Electric vehicles 2023                                       61840.902344   
                  2030                                           236409.0   
                  2035                                           247364.0   
                  2040                                           278864.0   
                  2045                                           313127.0   

                        demand_cobalt_refinery_base_case_stated_policies  \
country           year                                                     
Electric vehicles 2023                                      61840.902344   
                  2030                                          150718.0   
                  2035                                          160685.0   
                  2040                                          187206.0   
                  2045                                          203124.0   

                        demand_copper_refinery_base_case_announced_pledges  \
country           year                                                       
Electric vehicles 2023                                           395950.0    
                  2030                                          1870160.0    
                  2035                                          3256050.0    
                

In [19]:
from etl.grapher import helpers as gh
from etl.config import OWIDEnv

branch = 'performance-grapher'

owid_env = OWIDEnv.from_staging(branch)

tab = gh._adapt_table_for_grapher(table, engine=owid_env.engine)

# generate table with entity_id, year and value for every column
tables = list(gh._yield_wide_table(tab, na_action="drop"))

len(tables)

381

In [5]:
from etl.grapher.to_db import upsert_table, upsert_origins, DatasetUpsertResult, load_dataset_variables
from apps.chart_sync.admin_api import AdminAPI
import time
import pandas as pd
from sqlalchemy.orm import Session

from etl import config

admin_api = AdminAPI(owid_env)

engine = owid_env.engine

q = f"""
select * from datasets where catalogPath like '%%{step}%%'
"""
dataset_id = pd.read_sql(q, engine).iloc[0].id

dataset_upsert_results = DatasetUpsertResult(dataset_id=dataset_id, source_ids={})
catalog_path = 'grapher/' + step

preloaded_checksums = load_dataset_variables(dataset_upsert_results.dataset_id, engine)

with Session(engine, expire_on_commit=False) as session:
    db_origins = upsert_origins(session, table)

def foo(tables, force_upload=False):
    config.FORCE_UPLOAD = force_upload
    for tab in tables:
        catalog_path = f"{step}/{table_name}#{tab.columns[0]}"
        upsert_table(
            engine,
            admin_api,
            tab,
            dataset_upsert_results,
            catalog_path=catalog_path,
            checksums=preloaded_checksums.get(catalog_path, {}),
            db_origins=[db_origins[origin] for origin in tab.iloc[:, 0].origins],
        )

foo(tables[:1], force_upload=True)

2025-03-31 06:13:08 [info     ] upsert_table.uploaded_to_s3    size=12 variable=demand_graphite__battery_grade__refinery_base_case_announced_pledges


In [12]:
from etl.grapher import model as gm
from etl.grapher.to_db import upsert_metadata

# %lprun -f upsert_table foo(tables[:3], force_upload=True)
# %lprun -f upsert_metadata foo(tables[:3], force_upload=True)
# %lprun -f upsert_table foo(tables[:10], force_upload=True)
%lprun -f gm.Variable.update_links foo(tables[:3], force_upload=True)

# %lprun -f gm.Variable.from_catalog_path foo(tables[:10], force_upload=False)

2025-03-31 06:40:18 [info     ] upsert_table.uploaded_to_s3    size=12 variable=demand_graphite__battery_grade__refinery_base_case_announced_pledges
2025-03-31 06:40:19 [info     ] upsert_table.uploaded_to_s3    size=12 variable=demand_graphite__battery_grade__refinery_base_case_net_zero_by_2050
2025-03-31 06:40:19 [info     ] upsert_table.uploaded_to_s3    size=12 variable=demand_graphite__battery_grade__refinery_base_case_stated_policies


Timer unit: 1e-09 s

Total time: 0.469017 s
File: /Users/mojmir/projects/etl2/etl/grapher/model.py
Function: update_links at line 1420

Line #      Hits         Time  Per Hit   % Time  Line Contents
  1420                                               def update_links(
  1421                                                   self, session: Session, db_origins: List["Origin"], faqs: List[catalog.FaqLink], tag_names: List[str]
  1422                                               ) -> None:
  1423                                                   """
  1424                                                   Establishes relationships between the current variable and a list of origins and a list of posts.
  1425                                                   """
  1426         3      15000.0   5000.0      0.0          assert self.id
  1427                                           
  1428                                                   # Establish relationships between variables and ori